In [ ]:
import pandas as pd
import os, re
import numpy as np
import soundfile as sf
from IPython.display import clear_output
import pickle
import librosa
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras import Model
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import det_curve, DetCurveDisplay
import plotly.express as px
import plotly
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Przygotowanie danych do cross-checkingu oraz sam proces cross-checkingu.

In [ ]:
model = load_model("/kaggle/input/ubm_cnn/keras/default/1/model.h5");

with open("/kaggle/input/scaler-preprocessing/scaler.pkl", "rb") as file:
    scaler_before_embedding = pickle.load(file)
    
with open("/kaggle/input/scaler-and-lda-postprocessing/scaler_after_embedding.pkl", "rb") as file:
    scaler_after_embedding = pickle.load(file)

with open("/kaggle/input/scaler-and-lda-postprocessing/lda.pkl", "rb") as file:
    lda = pickle.load(file)

In [ ]:
def split_audio_to_slices(path_to_files, seconds):
    
    # Przechodzę do katalogów wewnątrz folderu osoby (ID osoby).
    # Każdy folder wewnętrzny zawiera więcej podfolderów, które mogą zawierać nagrania.
    paths_inside_ID = [f.name for f in os.scandir(path_to_files) if f.is_dir()]

    # Tworzę pełne ścieżki do podfolderów, aby przejść do wszystkich plików nagrań dla danej osoby.
    full_paths_to_files = [path_to_files + '/' + path_inside_ID for path_inside_ID in paths_inside_ID]

    # Zbieram wszystkie ścieżki do plików audio danej osoby.
    # Każdy plik powinien mieć rozszerzenie `.flac`, a wszystkie pliki są przechowywane w zmiennej `all_files_for_ID`.
    
    all_files_for_ID = []
    for full_path_to_files in full_paths_to_files:
        files = [f.name for f in os.scandir(full_path_to_files) if f.is_file() and f.name.endswith('.flac')]
        files = [full_path_to_files + '/' + file for file in files]
        all_files_for_ID = all_files_for_ID + files

    # Łączę wszystkie nagrania danej osoby w jedno bardzo długie nagranie.
    # Używam częstotliwości próbkowania 16kHz.
    sr = 16000
    combined_signals = np.array([])
    for file_for_ID in all_files_for_ID:
        signal, sr = librosa.load(file_for_ID, sr=sr)
        combined_signals = np.concatenate([combined_signals, signal])

    # Długie nagranie dzielę na  fragmenty o podanej długości.
    # Fragmenty, które mają mniej niż zadeklarowane długości nagrania (resztki na końcu nagrania), są pomijane.
    list_for_parts = []
    len_of_combined_signals = len(combined_signals)
    step = seconds * sr  # Ustawienie skoku na 5 sekund
    for i in np.arange(start=0, stop=len_of_combined_signals-step, step=step):
        list_for_parts.append(combined_signals[i:i+step].tolist())

    
    parts = np.array(list_for_parts)
    
    
    # Funkcja zwraca pociętę na kawałki, o długości 1 sekundy nagrania
    return parts
    
    


In [ ]:
class EbeddingExtractor:
    
    def __init__(self, model, scaler_before_embedding, scaler_after_embedding, lda):
        self.model = model  # Przypisanie modelu do obiektu
        self.lda = lda  # Przypisanie modelu LDA do obiektu
        self.scaler_before_embedding = scaler_before_embedding  # Skaler do standaryzacji MFCC przed generowaniem embeddingów
        self.scaler_after_embedding = scaler_after_embedding  # Skaler do standaryzacji embeddingów przed zastosowaniem LDA

    # Funkcja obliczająca współczynniki MFCC dla danego nagrania audio
    def calucate_MFCC(self, audio):
        quantity_of_mel_coef = 13  # Liczba współczynników MFCC
        quantity_of_mel_filters = 26  # Liczba filtrów Mel

        # Obliczanie współczynników MFCC za pomocą librosa
        mfcc = librosa.feature.mfcc(y=audio, 
                                    sr=16000, 
                                    n_mfcc=quantity_of_mel_coef, 
                                    n_mels=quantity_of_mel_filters).T
        
        # Standaryzacja MFCC przed generowaniem embeddingów
        mfcc = self.scaler_before_embedding.transform(mfcc)
        return mfcc

    # Funkcja obliczająca embedding na podstawie wcześniej przetworzonych MFCC
    def calcuate_embedding(self, audio_MFCC):
        
        intermediate_layer_model = Model(inputs=self.model.inputs,
                                         outputs=self.model.get_layer('dense_1').output)
        intermediate_output = intermediate_layer_model.predict(audio_MFCC[np.newaxis, ...])
        
        return intermediate_output

    # Funkcja postprocessingu embeddingu – standaryzacja i redukcja wymiarów za pomocą LDA
    def transform_audio_postprocessing(self, embedding):
        embedding = self.scaler_after_embedding.transform(embedding)  # Standaryzacja embeddingu
        embedding = self.lda.transform(embedding)  # Użycie LDA
        
        return embedding

    # Funkcja łączy wszystkie poprzednie umożliwiając dokonanie wyboru, czy przeprowadzony 
    # ma być postprocessing
    def process_audio_to_embedding(self, audio, use_lda):
        mfcc = calucate_MFCC(audio)
        embedding = calcuate_embedding(mfcc)
        if use_lda:
            embedding = transform_audio_postprocessing(embedding)

        return embedding

    

# Tworzenie obiektu klasy EmbeddingExtractor z przekazaniem modelu, skalerów i modelu LDA
EmbExtr = EbeddingExtractor(model=model, 
                            scaler_before_embedding=scaler_before_embedding,
                            scaler_after_embedding=scaler_after_embedding,
                            lda=lda)


In [ ]:
# Wczytanie danych z plików CSV do DataFrame dla mężczyzn i kobiet
data_frame_for_duration_man = pd.read_csv('/kaggle/input/data-to-cross-checking/data_frame_for_duration_man.csv')
data_frame_for_duration_woman = pd.read_csv('/kaggle/input/data-to-cross-checking/data_frame_for_duration_woman.csv')

# Wybranie próbek z określonego zakresu (wiersze 50 do 75) dla mężczyzn i kobiet do celów cross-checkingu
# Pierwsze 50 mężczyzn i kobiet zostało użyte do treningu UBM
man_to_cross_checking = data_frame_for_duration_man[50:75]
woman_to_cross_checking = data_frame_for_duration_woman[50:75]

# Definiowanie ścieżki do folderu z nagraniami
folders_path = '/kaggle/input/audio-to-train-model/train-clean-100'

# Tworzenie listy ścieżek do plików audio dla wybranych ID mężczyzn
man = [folders_path + '/' + str(ID) for ID in man_to_cross_checking['ID']]
# Tworzenie listy ścieżek do plików audio dla wybranych ID kobiet
woman = [folders_path + '/' + str(ID) for ID in woman_to_cross_checking['ID']]

# Połączenie list mężczyzn i kobiet w jedną listę `data_to_cross_checking`
data_to_cross_checking = man + woman

# Zapis listy ścieżek `data_to_cross_checking` do pliku pickle
with open("data_to_cross_checking.pkl", "wb") as file:
    pickle.dump(data_to_cross_checking, file)


In [ ]:
with open("/kaggle/input/data-to-enrollment-and-test/data_to_cross_checking.pkl", "rb") as file:
    data_to_cross_checking = pickle.load(file)

In [ ]:
embedding_all_people = []  # Lista do przechowywania embeddingów dla wszystkich osób

# Pętla przetwarzająca nagrania dla każdej z 50 osób
for i in range(0, 50):
    
    # Podział nagrania danej osoby na fragmenty 1-sekundowe za pomocą funkcji split_audio_to_slices
    slices = split_audio_to_slices(data_to_cross_checking[i], seconds=1)
    embedding_one_person = []  # Lista do przechowywania embeddingów dla pojedynczej osoby

    iterator = 0  # Inicjalizacja iteratora do śledzenia postępu przetwarzania
    for slice in slices:

        # Przekazywane jest audio o długości 1 s. a funkcja liczy MFCC, tworzy embedding. 
        # Przekazywana wartość logiczna mówi czy wykonany ma być postprocessing, czy nie.
        postprocessed_embedding = EmbExtr.process_audio_to_embedding(slice, T)
        
        #Wersja bez postprocessingu
        #embedding = EmbExtr.process_audio_to_embedding(slice, F)
        
        # Dodanie przetworzonego embeddingu do listy embeddingów danej osoby
        embedding_one_person.append(postprocessed_embedding)
        #embedding_one_person.append(embedding)
        
        # Aktualizacja iteratora i wyświetlanie postępu przetwarzania
        iterator += 1
        print(i + iterator / len(slices))  # Procentowy postęp przetwarzania dla osoby o ID i
        clear_output(wait=True)
    
    # Dodanie listy embeddingów danej osoby do głównej listy embeddingów dla wszystkich osób
    embedding_all_people.append(embedding_one_person)


In [ ]:
with open('embedding_all_people_without_postprocess.pkl', 'wb') as file:
    pickle.dump(embedding_all_people, file)

In [ ]:
#with open("/kaggle/input/embedding-all-people/embedding_all_people.pkl", "rb") as file:
#    embedding_all_people = pickle.load(file)

In [ ]:
#Embedding enrollment ma być stworzony z 40 s nagrania


emrollment_embedding_all_people = []  # Lista do przechowywania embeddingów enrollment dla wszystkich osób
test_embedding_all_people = []  # Lista do przechowywania embeddingów testowych dla wszystkich osób

# Pętla przetwarzająca embeddingi dla każdej osoby w embedding_all_people
for i in range(0, len(embedding_all_people)):
    
    # Inicjalizacja zerowych embeddingów dla enrollment i testowych (o długości 64)
    enrollment_embedding = np.zeros(64)
    test_embedding = np.zeros(64)
    test_embedding_one_person = []  # Lista do przechowywania testowych embeddingów dla jednej osoby

    # Zaokrąglenie liczby fragmentów danej osoby do najbliższej wielokrotności 10
    rounded_len = np.floor_divide(len(embedding_all_people[i]), 10) * 10
    
    for j in range(0, rounded_len):
        
        # Dodawanie embeddingów do enrollment dla pierwszych 40 fragmentów - enrollment ma być zbudowany z 
        #40 s nagrania
        if j < 40:
            enrollment_embedding += embedding_all_people[i][j][0]
        
        # Po co dziesiątym fragmencie: obliczenie średniego embeddingu testowego i resetowanie `test_embedding`
        # embedding test ma mieć długośc 10 s
        elif (j + 1) % 10 == 0:
            test_embedding = test_embedding / 10
            test_embedding_one_person.append(test_embedding)  # Dodanie średniego embeddingu do listy
            test_embedding = np.zeros(64)  # Reset embeddingu testowego do kolejnych obliczeń
        
        # Dodawanie embeddingów do testowego dla kolejnych fragmentów
        else:
            test_embedding += embedding_all_people[i][j][0]
    
    # Obliczenie średniego embeddingu enrollmentowego (średnia z 40 fragmentów)
    enrollment_embedding = enrollment_embedding / 40

    # Dodanie średniego embeddingu enrollment i listy testowych embeddingów do głównych list
    emrollment_embedding_all_people.append(enrollment_embedding)
    test_embedding_all_people.append(test_embedding_one_person)

    # Wyświetlanie postępu przetwarzania dla każdej osoby
    print((i + 1) / len(embedding_all_people))
    clear_output(wait=True)


In [ ]:
# DataFrame do przechowywania wyników
data_rows = []
data_rows_trans = []

for i in range(0, len(emrollment_embedding_all_people)):
    embedding_enrollment = emrollment_embedding_all_people[i].reshape(1, -1)

    for j in range(0, len(test_embedding_all_people)):
        # Wczytanie embeddingów testowych dla danej osoby
        one_person_list = np.array(test_embedding_all_people[j])
        random_numbers = np.random.uniform(0, 1, len(one_person_list)) > 0.5
        one_person_list = one_person_list[random_numbers]

        for h in range(0, len(one_person_list)):
            # Wczytanie embeddingu testowego
            embedding_eval = one_person_list[h].reshape(1, -1)
            # Obliczenie podobieństwa kosinusowego
            cos_sim = cosine_similarity(embedding_enrollment, embedding_eval)[0][0]
            # Zapisanie wyników
            data_rows.append([i, j, i == j, cos_sim])

    clear_output(wait=True)
    print(f"Processing ID: {i}")

# Tworzenie DataFrame z listy wyników
long_data_frame = pd.DataFrame(data_rows, columns=['ID_enrollment', 'ID_test', 'genuine', 'score'])

In [ ]:
long_data_frame.to_csv('long_data_frame_without_postprocess.csv', index=False)